In [141]:
import pandas as pd

# install Pint if necessary
try:
    from pint import UnitRegistry
except ImportError:
    !pip install pint

In [142]:
# download modsim.py if necessary

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'modsim.py')

In [143]:
# Menghapus kolom yang tidak diperlukan
data_have_index = [
    "sample/anxiety-disorders-males-vs-females.csv",
    "sample/bipolar-disorder-in-males-vs-females.csv",
    "sample/depression-males-vs-females.csv",
    "sample/eating-disorders-in-males-vs-females.csv",
    "sample/Mental health Depression disorder Data.csv",
    "sample/schizophrenia-in-males-vs-females.csv"
]

for file_path in data_have_index:
    df = pd.read_csv(file_path)
    if 'index' in df.columns:
        df = df.drop('index', axis=1)
    df.to_csv(file_path, index=False)

C:\Users\katana\AppData\Local\Temp\ipykernel_18968\2974179843.py:12: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [144]:
df_grouped_bundir = pd.read_csv("sample/suicide-rates-among-young-people.csv")
df_grouped_bundir = df_grouped_bundir.groupby('Year').mean().reset_index()

C:\Users\katana\AppData\Local\Temp\ipykernel_18968\1967768737.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped_bundir = df_grouped_bundir.groupby('Year').mean().reset_index()


In [145]:
df_grouped_bundir.columns

Index(['Year',
       'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19',
       'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24',
       'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34'],
      dtype='object')

In [146]:
df_grouped_bundir['Change_15_19'] = df_grouped_bundir['Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19'].round()
df_grouped_bundir['Change_20_24'] = df_grouped_bundir['Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24'].round()
df_grouped_bundir['Change_25_34'] = df_grouped_bundir['Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34'].round()

print(df_grouped_bundir[['Year', 'Change_15_19', 'Change_20_24', 'Change_25_34']])

    Year  Change_15_19  Change_20_24  Change_25_34
0   2000           8.0          13.0          14.0
1   2001           8.0          13.0          14.0
2   2002           8.0          13.0          14.0
3   2003           7.0          12.0          13.0
4   2004           7.0          12.0          13.0
5   2005           7.0          12.0          13.0
6   2006           7.0          12.0          13.0
7   2007           7.0          12.0          13.0
8   2008           7.0          11.0          13.0
9   2009           7.0          11.0          13.0
10  2010           7.0          11.0          13.0
11  2011           7.0          11.0          13.0
12  2012           7.0          11.0          12.0
13  2013           7.0          10.0          12.0
14  2014           7.0          10.0          12.0
15  2015           7.0          10.0          12.0
16  2016           6.0          10.0          11.0
17  2017           6.0          10.0          11.0
18  2018           6.0         

In [147]:
df_grouped_bundir['Mean_Total'] = df_grouped_bundir[['Change_15_19', 'Change_20_24', 'Change_25_34']].sum(axis=1).round()

print(df_grouped_bundir[['Year', 'Change_15_19', 'Change_20_24', 'Change_25_34', 'Mean_Total']])

    Year  Change_15_19  Change_20_24  Change_25_34  Mean_Total
0   2000           8.0          13.0          14.0        35.0
1   2001           8.0          13.0          14.0        35.0
2   2002           8.0          13.0          14.0        35.0
3   2003           7.0          12.0          13.0        32.0
4   2004           7.0          12.0          13.0        32.0
5   2005           7.0          12.0          13.0        32.0
6   2006           7.0          12.0          13.0        32.0
7   2007           7.0          12.0          13.0        32.0
8   2008           7.0          11.0          13.0        31.0
9   2009           7.0          11.0          13.0        31.0
10  2010           7.0          11.0          13.0        31.0
11  2011           7.0          11.0          13.0        31.0
12  2012           7.0          11.0          12.0        30.0
13  2013           7.0          10.0          12.0        29.0
14  2014           7.0          10.0          12.0     

In [166]:
# Anxiety
df_anxiety = pd.read_csv("sample/anxiety-disorders-males-vs-females.csv")
# df_anxiety.head(20000)
# df_anxiety.columns

df_anxiety_filtered = df_anxiety[(df_anxiety['Year'] >= 2000) & (df_anxiety['Year'] <= 2019)]

df_anxiety_mean = df_anxiety_filtered.groupby('Year').agg({
    'Prevalence - Anxiety disorders - Sex: Male - Age: Age-standardized (Percent)': 'mean',
    'Prevalence - Anxiety disorders - Sex: Female - Age: Age-standardized (Percent)': 'mean'
})

total_df_anxiety_mean =  df_anxiety_mean['Prevalence - Anxiety disorders - Sex: Female - Age: Age-standardized (Percent)'] + df_anxiety_mean['Prevalence - Anxiety disorders - Sex: Male - Age: Age-standardized (Percent)']

total_df_anxiety_mean.round()


Year
2000    9.0
2001    9.0
2002    9.0
2003    9.0
2004    9.0
2005    9.0
2006    9.0
2007    9.0
2008    9.0
2009    9.0
2010    9.0
2011    9.0
2012    9.0
2013    9.0
2014    9.0
2015    9.0
2016    9.0
2017    9.0
2018    9.0
2019    9.0
dtype: float64